In [2]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
# loading the ann trained model and pickle files

model = load_model('model.h5')

# loading the pickle files

with open('geo_ohencoder.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file2:
    label_encoder_gender = pickle.load(file2)

with open('scaler.pkl', 'rb') as file3:
    scaler = pickle.load(file3)

In [4]:
# example input data to make a prediction, later this will be give via streamlit

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
# first we need to convert categorical values and then apply the scaler to the input data

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/domasbelickas/git_repos/GenAI/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
# transform input_data to df

input_data_df = pd.DataFrame([input_data])

In [13]:
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
# concating with onehotencdoing data

input_data_df = pd.concat([input_data_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [18]:
input_data_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
# scaling the input data

input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [20]:
# so we need to predict 

prediction = model.predict(input_scaled)
print(prediction)

1/1 [==============================] - 0s 329ms/step
[[0.02350322]]


In [21]:
# probability
prediction[0][0]

0.023503218